# Transformer training (chantier en cours)

Imports

In [1]:
%load_ext autoreload
%autoreload 2

from transformer_model import *
import nltk
import sys
sys.path.append("../../..")

In [5]:
from statapp.common.preprocessing import load_all_data, encode_data, split_into_X_y

In [6]:
from statapp.common.sampling import sample_token_sequence

Preprocessing maison bien moche pour le moment... ^^ (essai avec un encodage sur les mots) Les données sont placées directement dans le dossier du notebook

In [7]:
text = load_all_data("data/fr.train.top1M.txt", sample=0.0000006)

tokens = nltk.word_tokenize(text)

In [8]:
len(tokens)

36

In [198]:
vocab = list(set(tokens))

vocab_size = len(vocab)

vocab_numbers = dict(zip(vocab, range(0,len(vocab))))
vocab_numeroted = dict(zip(range(0,len(vocab)), vocab))
tokens_numbers = np.array([vocab_numbers[tokens[i]] for i in range(len(tokens))])

tokens_numbers_sequences = np.array([ tokens_numbers[i:i+max_length] for i in range(len(tokens_numbers)-max_length+1)])
tokens_numbers_sequences = torch.tensor(tokens_numbers_sequences , dtype=torch.int64)

nb_sequences =  tokens_numbers_sequences.shape[0]

In [199]:
vocab_size

27

Apprentissage du modèle

In [200]:
LMtransformer = Transformer(vocab_size, Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

#Correspond à utiliser l'entropie croisée puisque les sorties sont des log_softmax
#et l'entropie croisée = nll_loss(log_softmax(.), target)
criterion = nn.NLLLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.01)

In [201]:
def train_model(nb_epochs, batch_size):
    
    #What is this ?? I don't remember. Make grad required ?
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        randperm = torch.randperm(nb_sequences)
        randperm = randperm[:(nb_sequences//batch_size)*batch_size]
        batchs_indices = randperm.reshape(nb_sequences//batch_size, batch_size)
        
        for i, batch_indices in enumerate(batchs_indices):
            
            batch = tokens_numbers_sequences[batch_indices]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            loss = criterion(output[:,:-1].reshape(-1, vocab_size), batch[:,1:].flatten())

            loss.backward()
            optimizer.step()
            
            #Il faudrait adapter les affichages en fonction du nombre de batchs total
            running_loss += loss.item()
            #if i % 10 == 9:
            if i:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.

Test d'overfitting sur un cas ultrasimplifié (5 tokens, longueur de séquence 1, 3 decoders, 2 heads) :
- En observant les sorties le modèle a bien appris et overfitte ! (loss à 0 au bout de 5-6 epochs) 

In [223]:
train_model(10,10)

[1,     2] loss: 0.001
[2,     2] loss: 0.001
[3,     2] loss: 0.001
[4,     2] loss: 0.001
[5,     2] loss: 0.001
[6,     2] loss: 0.001
[7,     2] loss: 0.001
[8,     2] loss: 0.001
[9,     2] loss: 0.001
[10,     2] loss: 0.001


Génération

In [206]:
#bidouilles pour adapter nos fonctions aux fonctions common codées par Nathra 
#(sequence list of ints en entree, list of probas en sortie)
#(Faire mieux plus tard)
def LMtransformerprediction(listints):
    return np.exp(LMtransformer(torch.tensor([listints]))[0][-1].tolist())

In [225]:
print(vocab)

['nom', 'du', 'touche', 'obtenu', 'l', 'connaissance', 'il', 'barcelone', 'age', 'de', '.', 'alors', 'ans', 'apres', 'vrai', ',', 'la', 'dieu', 'a', '31', 'par', 'persuade', "'", 'prophetique', 'est', 'esprit', 'avoir']


In [230]:
prev_seq = ['a','l','age','de']
prev_seq_numbers = [vocab_numbers[token] for token in prev_seq]
i=np.argmax(LMtransformerprediction(prev_seq_numbers))
tokens_pred = vocab_numeroted[i] 
print(tokens_pred)

31


In [207]:
def gen_seq(prev_seq):
    with torch.no_grad():
        prev_seq_numbers = [vocab_numbers[token] for token in prev_seq]
        sample_token_seq = sample_token_sequence(LMtransformerprediction, prev_seq_numbers)
        tokens_pred = [vocab_numeroted[i] for i in sample_token_seq]
        print(' '.join(tokens_pred)) 

In [ ]:
def gen_seq_maison(prev_seq):
    with torch.no_grad():
        prev_seq_numbers = [vocab_numbers[token] for token in prev_seq]
        sample_token_seq = sample_token_sequence(LMtransformerprediction, prev_seq_numbers)
        tokens_pred = [vocab_numeroted[i] for i in sample_token_seq]
        print(' '.join(tokens_pred))

In [208]:
gen_seq(['il'])

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 132.45it/s]


est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est alors persuade de dieu . il est alors persuade la connaissance du vrai nom de dieu . il est alors persuade de obtenu la connaissance du vrai nom de dieu . il est alors persuade de dieu . il est alors persuade de dieu . il est alors prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est alors persuade avoir obtenu la connaissance du vrai nom de dieu . il est alors persuade prophetique apres


In [242]:
gen_seq(['a','l','age','de','31','ans'])

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 132.45it/s]


, a barcelone , il est touche par l ' age de dieu . il est touche par l ' age de dieu . il est touche par l ' age de dieu . il est touche par l ' age de dieu . il est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai apres avoir obtenu la connaissance du vrai nom de dieu . il est touche par l ' age de dieu . il est touche par l ' age de dieu . il est touche par l ' age de dieu .


In [245]:
gen_seq(['barcelone',',','il','est','touche','par','l'])

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 132.63it/s]


' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu touche avoir obtenu la connaissance du vrai nom de dieu . il est touche par connaissance du vrai nom de dieu . il est touche par connaissance du vrai nom de dieu . il la connaissance du vrai nom de dieu . il est alors persuade de dieu . il est alors persuade avoir obtenu la connaissance du vrai nom de dieu . il est alors persuade de dieu . il est alors persuade la connaissance du vrai nom de obtenu la connaissance du vrai nom de


In [210]:
print(' '.join(tokens))

a l ' age de 31 ans , a barcelone , il est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est alors persuade
